In [ ]:
%pip install pandas sqlalchemy langchain langchain_community langchain_openai python_dotenv  --default-timeout=100

In [ ]:
import pandas as pd

df = pd.read_csv("./F1.csv")

# Creación de base de datos SQL a partir de dataframe (csv)

In [ ]:
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///f1.db', echo=True)
df.to_sql('f1', con=engine, if_exists='replace', index=False)

# Creación de agente
- La temperatura es 0 para que se limite a responder con la información de la base de datos y que no se ponga creativo

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

db = SQLDatabase(engine=engine)

llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))

agente = create_sql_agent(llm=llm, database=db,agent_type="openai-tools" , verbose=True) #verbose=True para ver como "piensa" el agente

# Algunos ejemplos de consultas

In [ ]:
response = agente.invoke({"input":"¿Cuántos pilotos que ganaron mas de campeonato mundial hay en la base de datos?"})
print(response['output'])

In [ ]:
response = agente.invoke({"input":"¿Cuántos campeonatos mundiales tiene Ayrton Senna?"})
print(response['output'])

In [ ]:
response = agente.invoke({"input":"¿Qué porcentaje de carreras se corrieron bajo la lluvia?"})
print(response['output'])

In [ ]:
response = agente.invoke({"input":"¿Cómo se relaciona la posicion final con los puntos obtenidos?"})
print(response['output'])